In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

np.random.seed(17)
tf.random.set_seed(17)

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data() 

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step 
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 2s 1us/step 


In [4]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [6]:
val_frac = 0.25

x_val = x_train[round((1-val_frac) * len(x_train)):]
y_val = y_train[round((1-val_frac) * len(y_train)):]

x_train = x_train[:round((1-val_frac) * len(x_train))]
y_train = y_train[:round((1-val_frac) * len(y_train))]

In [7]:
# Перебираем значения дропаута
best_dropout = 0.0
best_accuracy = 0.0

dropout_values = [0.2, 0.3, 0.4, 0.5]

for drp in dropout_values:
    print(f"\nТестируем dropout = {drp}")


Тестируем dropout = 0.2

Тестируем dropout = 0.3

Тестируем dropout = 0.4

Тестируем dropout = 0.5


In [9]:
# Создаем модель
model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(300, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(drp),
        tf.keras.layers.Dense(300, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(drp),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [11]:
# Компилируем модель
model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])

In [12]:
# Обучаем модель
history = model.fit(x_train, y_train,
                       epochs=10,
                       batch_size=128,
                       validation_data=(x_val, y_val),
                       verbose=0)

In [14]:
# Получаем точность на валидации
val_accuracy = history.history['val_accuracy'][-1]
print(f"Точность на валидации: {val_accuracy:.4f}")

Точность на валидации: 0.8852


In [15]:
# Обновляем лучшее значение
if val_accuracy > best_accuracy:
    best_accuracy = val_accuracy
    best_dropout = drp

In [16]:
print(f"\nОптимальное значение dropout: {best_dropout}")
print(f"Лучшая точность: {best_accuracy:.4f}")


Оптимальное значение dropout: 0.5
Лучшая точность: 0.8852


In [17]:
# Создаем модель с оптимальным dropout=0.4
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [18]:
# Компилируем модель
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [19]:
# Создаем коллбэк EarlyStopping
stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=4,
    verbose=1
)

In [20]:
# Обучаем модель с EarlyStopping
history = model.fit(
    x_train, y_train,
    epochs=50,
    batch_size=128,
    validation_data=(x_val, y_val),
    callbacks=[stopping],
    verbose=1
)

Epoch 1/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7827 - loss: 0.6333 - val_accuracy: 0.8541 - val_loss: 0.4208
Epoch 2/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8411 - loss: 0.4433 - val_accuracy: 0.8637 - val_loss: 0.3762
Epoch 3/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8545 - loss: 0.3959 - val_accuracy: 0.8700 - val_loss: 0.3566
Epoch 4/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8673 - loss: 0.3645 - val_accuracy: 0.8763 - val_loss: 0.3422
Epoch 5/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8761 - loss: 0.3453 - val_accuracy: 0.8769 - val_loss: 0.3375
Epoch 6/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8804 - loss: 0.3294 - val_accuracy: 0.8777 - val_loss: 0.3360
Epoch 7/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8844 - loss: 0.3122 - val_accuracy: 0.8808 - val_loss: 0.3238
Epoch 8/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8873 - loss: 0.3010 - val_accuracy: 0.

In [21]:
# Оцениваем модель на тестовой выборке
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)

print(f"\nТочность на тестовой выборке: {test_accuracy:.4f}")
print(f"Точность на тестовой выборке (округлено до сотых): {round(test_accuracy, 2)}")


Точность на тестовой выборке: 0.8848
Точность на тестовой выборке (округлено до сотых): 0.88
